In [1]:
# импорт библиотек
import warnings

warnings.simplefilter('ignore', UserWarning)

# import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import spacy
import numpy as np

from tqdm import tqdm
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# from itertools import product

ModuleNotFoundError: No module named 'seaborn'

In [2]:
sns.set_style('whitegrid')

In [3]:
# загрузка модели для обработки текста
nlp = spacy.load('ru_core_news_lg')

In [4]:
# import data
ozon_cats = pd.read_csv('bazungaru_platfo_table_ozon_categories.csv', names=['id', 'title', 'pid'])
ozon_chars = pd.read_csv('bazungaru_platfo_table_ozon_characteristics.csv',
                         names=['description', 'dictionary_id', 'group_id', 'group_name', 'id', 'is_collection',
                                'is_required', 'name', 'type', 'category_id'])

wb_cats = pd.read_csv('bazungaru_platfo_table_wb_categories.csv', names=['id', 'pid', 'title'])
wb_chars = pd.read_csv('bazungaru_platfo_table_wb_characteristics.csv',
                       names=['id', 'pid', 'object_name', 'charcid', 'name', 'required', 'unitName', 'maxCount',
                              'popular', 'aocid', 'aocName', 'charcType', 'allowUserVals', 'sort'])

/var/folders/69/5q4hf9x56rb_wd5yv511rrt80000gn/T/ipykernel_5359/3596266331.py:8: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  wb_chars = pd.read_csv('bazungaru_platfo_table_wb_characteristics.csv',


In [5]:
ozon_text = ozon_cats['title'].drop_duplicates().reset_index(drop=True).apply(lambda x: x.lower()).to_list()

wb_text = wb_cats['title'].drop_duplicates().reset_index(drop=True).apply(lambda x: x.lower()).to_list()

In [6]:
ozon_cats['title'] = ozon_cats['title'].apply(lambda x: x.lower())

wb_cats['title'] = wb_cats['title'].apply(lambda x: x.lower())

### Многоуровневая классификация

#### Ozon

In [7]:
ozon_merged = ozon_cats.merge(ozon_cats, left_on='id', right_on='pid').rename(columns={'id_x': 'second_id',
                                                                                       'id_y': 'third_id'}).merge(
    ozon_cats, left_on='pid_x', right_on='id').rename(columns={'title_x': 'second_name',
                                                               'title_y': 'third_name',
                                                               'title': 'first_name',
                                                               'id': 'first_id'})[
    ['first_id', 'second_id', 'third_id', 'first_name', 'second_name', 'third_name']]

#### Wildberries

In [8]:
wb_merged = wb_cats.merge(wb_chars, left_on='id', right_on='pid').rename(columns={'id_x': 'first_id',
                                                                                  'id_y': 'second_id',
                                                                                  'title': 'first_name',
                                                                                  'object_name': 'second_name'})[
    ['first_id', 'second_id', 'first_name', 'second_name']].drop_duplicates().reset_index(drop=True)

## Предобработка данных, вычисление "схожести"

In [9]:
def get_clear_text(x):
    doc = nlp(x)
    new_text = [token.lemma_ for token in doc if token.lemma_ not in nlp.Defaults.stop_words and not token.is_punct]
    vectors = all(token.vector_norm for token in doc)
    if vectors:
        return ' '.join(new_text)
    else:
        return 'null_vector'

In [ ]:
# списки из объектов spacy
ozon_nlp = list(nlp.pipe(ozon_text))
wb_nlp = list(nlp.pipe(wb_text))

### Первая итерация

Не используется предобработка текста

In [ ]:
def get_similarity(first_list, second_list, first_root_list=ozon_text, second_root_list=wb_text):
    df = pd.DataFrame()

    for i in tqdm(range(len(first_list))):
        l = np.array([first_list[i].similarity(k) for k in second_list])
        temp_df = pd.DataFrame({'first_list': first_root_list[i],
                                'second_list': second_root_list[l.argmax()],
                                'similarity': max(l)},
                               index=[i])

        df = pd.concat([df, temp_df])

    return df

In [58]:
df = get_similarity(ozon_nlp, wb_nlp)

  0%|          | 0/10346 [00:00<?, ?it/s]

In [60]:
def get_group(df):
    """
    Функция добавляет столбец в ДФ в зависимости от "схожести"
    :param df:  ДФ с метрикой "схожести"
    """
    # q = df['similarity'].quantile(0.5)
    q = 0.6
    df.loc[df['similarity'] >= 1, 'group'] = 'Точное совпадение'
    df.loc[(df['similarity'] > q) & (df['similarity'] < 1), 'group'] = 'Хорошее совпадение'
    df.loc[df['similarity'] <= q, 'group'] = 'Плохое совпадение'
    print('Results are grouped')

In [61]:
get_group(df)

Results are grouped


### Вторая итерация

В качестве обработки используются:
1. лемматизация слов;
2. удаление стоп-слов;
3. удаление пунктуации.

In [80]:
def preprocess_data(l):
    """
    Функция обрабатывает подаваемый список: лемматизирует слова, удаляем стоп-слова и пунктуацию.

    :param l: список из объектов spacy
    :return: обработанный список из объектов spacy, обработанный список из строк, список нулевых векторов
    """
    res_list_lemma = []
    res_list_text = []
    null_vectors_list = []

    for chunk in tqdm(range(len(l))):
        new_chunk_lemma = ' '.join(
            [token.lemma_ for token in l[chunk] if not token.is_punct and token.lemma_ not in nlp.Defaults.stop_words])
        new_chunk_text = ' '.join([token.text for token in l[chunk]])
        has_null_vector = int(not all([token.vector_norm for token in nlp(new_chunk_lemma)]))

        if has_null_vector:
            null_vectors_list.append(l[chunk])
        else:
            res_list_lemma.append(new_chunk_lemma)
            res_list_text.append(new_chunk_text)

    return list(nlp.pipe(res_list_lemma)), res_list_text, set(null_vectors_list)

In [66]:
ozon_second_lemma, ozon_second_text, ozon_second_null = preprocess_data(ozon_nlp)
wb_second_lemma, wb_second_text, wb_second_null = preprocess_data(wb_nlp)

  0%|          | 0/10346 [00:00<?, ?it/s]

  0%|          | 0/7368 [00:00<?, ?it/s]

In [68]:
second_res = get_similarity(ozon_second_lemma, wb_second_lemma, ozon_second_text, wb_second_text)
get_group(second_res)

  0%|          | 0/7773 [00:00<?, ?it/s]

Results are grouped


### Третья итерация

В качестве обработки используются:
1. лемматизация слов;
2. удаление стоп-слов;
3. удаление пунктуации;
4. сравниваются только одинаковые по количеству слов характеристики.

In [ ]:
def get_similarity_len(first_list, second_list):
    df = pd.DataFrame()

    for i in tqdm.notebook.tqdm_notebook(range(len(first_list))):
        l = np.array([first_list[i].similarity(k) for k in second_list if len(first_list[i]) == len(k)])
        try:
            temp_df = pd.DataFrame({'first_list': first_list[i].text,
                                    'second_list': second_list[l.argmax()].text,
                                    'similarity': max(l)},
                                   index=[i])
        except:
            temp_df = pd.DataFrame({'first_list': first_list[i].text,
                                    'second_list': -1,
                                    'similarity': -1},
                                   index=[i])

        df = pd.concat([df, temp_df])

    return df

In [ ]:
third_res = get_similarity_len(ozon_second, wb_second)

### Четвёртая итерация

В качестве обработки используются:
1. удаление стоп-слов;
2. удаление пунктуации

In [ ]:
def preprocess_data_text(l):
    """
    Args:
        l (list): list of spacy objects

    Returns:
        list of preprocessed objects and list of objects with null vectors
    """

    res_list = []
    null_vectors_list = []

    for chunk in tqdm.notebook.tqdm_notebook(range(len(l))):
        new_chunk = ' '.join(
            [token.text for token in l[chunk] if not token.is_punct and token.lemma_ not in nlp.Defaults.stop_words])
        has_null_vector = int(not all([token.vector_norm for token in nlp(new_chunk)]))

        if has_null_vector:
            null_vectors_list.append(l[chunk])
        else:
            res_list.append(new_chunk)

    return list(nlp.pipe(res_list)), null_vectors_list

In [ ]:
ozon_prep_text, ozon_nulls = preprocess_data_text(ozon_nlp)
wb_prep_text, wb_null = preprocess_data_text(wb_nlp)

In [ ]:
fourth_res = get_similarity(ozon_prep_text, wb_prep_text)

In [ ]:
fourth_res.sort_values('similarity', ascending=False)

### Пятая итерация
Предобработка не осуществляется

In [ ]:
def preprocess_data_5(l):
    """
    Args:
        l (list): list of spacy objects

    Returns:
        list of preprocessed objects and list of objects with null vectors
    """

    res_list = []
    null_vectors_list = []

    for chunk in tqdm.notebook.tqdm_notebook(range(len(l))):
        new_chunk = ' '.join([token.text for token in l[chunk]])
        has_null_vector = int(not all([token.vector_norm for token in nlp(new_chunk)]))

        if has_null_vector:
            null_vectors_list.append(l[chunk])
        else:
            res_list.append(new_chunk)

    return list(nlp.pipe(res_list)), null_vectors_list

In [ ]:
ozon_fifth, _ = preprocess_data_5(ozon_nlp)
wb_fifth, _ = preprocess_data_5(wb_nlp)

In [ ]:
fifth_res = get_similarity(ozon_fifth, wb_fifth)

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(25, 13))
sns.boxplot(y=second_res['similarity'], ax=ax[0, 0])
sns.histplot(data=second_res, x='similarity', ax=ax[0, 1], hue='group')
sns.boxplot(y=fourth_res['similarity'], ax=ax[1, 0])
sns.histplot(data=fourth_res, x='similarity', ax=ax[1, 1], hue='group')

In [ ]:
second_res.query("group != 'Плохое совпадение'").sort_values('similarity')

### Шестая итерация

Схожесть вычисляется по двум уровням: где она выше, то значение и будет итогом.

In [104]:
test = get_similarity_v2(ozon_1_preprocessed, wb_1_preprocessed, wb_2_preprocessed, ozon_1_text, wb_1_text, wb_2_text)

  0%|          | 0/7069 [00:00<?, ?it/s]

In [9]:
# создание комплексных id
ozon_merged['complex_id'] = ozon_merged[['first_id', 'second_id', 'third_id']].astype(str).apply('_'.join, axis=1)
wb_merged['complex_id'] = wb_merged[['first_id', 'second_id']].astype(str).apply('_'.join, axis=1)

In [10]:
ozon_merged

,first_id,second_id,third_id,first_name,second_name,third_name,complex_id
0,29,52,17034470,печатные издания,букинистическая книга,букинистика,29_52_17034470
1,29,52,88044280,печатные издания,букинистическая книга,букинистика: детям и родителям,29_52_88044280
2,29,52,88044549,печатные издания,букинистическая книга,букинистика: комиксы,29_52_88044549
3,29,52,88044560,печатные издания,букинистическая книга,букинистика: манга,29_52_88044560
4,29,52,88044561,печатные издания,букинистическая книга,букинистика: бизнес-литература,29_52_88044561
...,...,...,...,...,...,...,...
9451,200000175,200000353,1000001834,питомцы,икра для разведения рыб,икра для разведения аквариумных рыб,200000175_200000353_1000001834
9452,200000175,200000847,1000003360,питомцы,пауки,паук (живой),200000175_200000847_1000003360
9453,200000175,200000848,1000003362,питомцы,ящерицы,ящерица (живая),200000175_200000848_1000003362
9454,200000175,200000851,1000003361,питомцы,змеи,змея (живая),200000175_200000851_1000003361


In [12]:
wb_merged

,first_id,second_id,first_name,second_name,complex_id
0,1,11,одежда,Брюки,1_11
1,1,38,одежда,Юбки,1_38
2,1,41,одежда,Блузки,1_41
3,1,69,одежда,Платья,1_69
4,1,70,одежда,Сарафаны,1_70
...,...,...,...,...,...
7295,7580,6477,мебель,Ламели для кровати,7580_6477
7296,7580,7430,мебель,Столешницы для кухни,7580_7430
7297,7580,6416,мебель,Домкраты для мебели,7580_6416
7298,7580,7071,мебель,Стойки ресепшн,7580_7071


In [12]:
def get_similarity_v2(df_1, df_2):
    # столбцы с наименованиями групп
    df_1_names = [name for name in df_1.columns if 'name' in name]
    df_2_names = [name for name in df_2.columns if 'name' in name]

    for i in tqdm(df_1_names):
        nlp_1, text_1, null_1 = preprocess_data(list(nlp.pipe(df_1[i].to_list())))
        for j in tqdm(df_2_names):
            nlp_2, text_2, null_2 = preprocess_data(list(nlp.pipe(df_2[i].to_list())))
            lim_list = [first_list[i].similarity(k) for k in second_list]


In [18]:
res_list = []

In [81]:
wb_list = list(nlp.pipe(wb_merged['first_name'].to_list()))

In [82]:
wb_preprocessed, wb_text, wb_nulls = preprocess_data(wb_list)

100%|██████████| 7300/7300 [00:51<00:00, 142.21it/s]


In [83]:
ozon_list = list(nlp.pipe(ozon_merged['first_name'].to_list()))

In [84]:
ozon_preprocessed, ozon_text, ozon_nulls = preprocess_data(ozon_list)

100%|██████████| 9456/9456 [01:04<00:00, 147.65it/s]


In [66]:
ozon_1 = list(nlp.pipe(ozon_merged['third_name'].unique()))
wb_1 = list(nlp.pipe(wb_merged['second_name'].unique()))

In [67]:
ozon_1_preprocessed, ozon_1_text, ozon_1_nulls = preprocess_data(ozon_1)
wb_1_preprocessed, wb_1_text, wb_1_nulls = preprocess_data(wb_1)

  0%|          | 0/9456 [00:00<?, ?it/s]

  0%|          | 0/7300 [00:00<?, ?it/s]

In [70]:
res_1 = get_similarity(ozon_1_preprocessed, wb_1_preprocessed, ozon_1_text, wb_1_text)

  0%|          | 0/7069 [00:00<?, ?it/s]

In [79]:
wb_2 = list(nlp.pipe(wb_merged['first_name'].unique()))
wb_2_preprocessed, wb_2_text, wb_2_nulls = preprocess_data(wb_2)

  0%|          | 0/68 [00:00<?, ?it/s]

In [82]:
res_2 = get_similarity(ozon_1_preprocessed, wb_2_preprocessed, ozon_1_text, wb_2_text)

  0%|          | 0/7069 [00:00<?, ?it/s]

In [83]:
res_2 = res_2.rename(columns={'second_list': 'second_list_second_try',
                              'similarity': 'second_similarity'})

In [85]:
res_1.merge(res_2, on='first_list').query('similarity < second_similarity')

,first_list,second_list,similarity,second_list_second_try,second_similarity
29,книги : страхование,Страхование техники,0.688830,страхование,0.739926
100,"книги : красота , здоровье и спорт",Книги,0.530251,здоровье,0.630474
257,мужская ортопедическая обувь,Дезодоранты для обуви,0.659245,спортивная обувь,0.680235
268,женская ортопедическая обувь,Карнавальная обувь,0.656805,спортивная обувь,0.659502
278,ботинки спортивные для девочки,Брюки спортивные,0.721188,спортивная обувь,0.723451
...,...,...,...,...,...
6822,по - для обработки фото и видео,Диски для записи,0.518316,фото и видеотехника,0.651780
6823,дистрибутив - для обработки фото и видео,Диски для записи,0.504064,фото и видеотехника,0.600119
6838,цифровой контент - офисное приложение,Видеокамеры цифровые,0.575407,сетевое оборудование,0.575619
6854,цифровой контент - игра для xbox,Картриджи игровые,0.564280,игровые консоли и игры,0.615449


## Расстояние Жаккарта

In [ ]:
ozon_prep, _ = preprocess_data(ozon_nlp)

In [ ]:
wb_prep, _ = preprocess_data(wb_nlp)

In [ ]:
def get_similarity_jak(first_list, second_list):
    df = pd.DataFrame()

    for i in tqdm(range(len(first_list))):
        a = set(str(first_list[i]).split())
        temp_arr = np.array([])
        for k in second_list:
            b = set(str(k).split())
            shared = len(a.intersection(b))
            total = len(a.union(b))
            try:
                jak = shared / total
            except:
                jak = 0

            temp_arr = np.append(temp_arr, jak)

        temp_df = pd.DataFrame({'first_list': first_list[i],
                                'second_list': second_list[temp_arr.argmax()],
                                'similarity': max(temp_arr)},
                               index=[i])

        df = pd.concat([df, temp_df])

    return df

In [ ]:
jak_res = get_similarity_jak(ozon_prep, wb_prep)

In [ ]:
get_group(jak_res)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(25, 13))
sns.boxplot(y=jak_res['similarity'], ax=ax[0])
sns.histplot(data=jak_res, x='similarity', ax=ax[1], hue='group')

In [ ]:
jak_res.sort_values('similarity', ascending=False)

# Rubert-tiny2

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [ ]:
def embed_bert_cls(text, model=model, tokenizer=tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [ ]:
def get_similarity_rubert(first_list, second_list):
    df = pd.DataFrame()

    for i in tqdm(range(len(first_list))):
        i_emb = embed_bert_cls(first_list[i].text)
        l = np.array([1 - cosine(i_emb, embed_bert_cls(k.text)) for k in second_list])

        temp_df = pd.DataFrame({'first_list': first_list[i].text,
                                'second_list': second_list[l.argmax()].text,
                                'similarity': max(l)},
                               index=[i])

        df = pd.concat([df, temp_df])

    return df

In [ ]:
rubert_res = get_similarity_rubert(ozon_prep, wb_prep)

In [ ]:
rubert_res.sort_values('similarity', ascending=False)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(25, 13))
sns.boxplot(y=rubert_res['similarity'], ax=ax[0])
sns.histplot(data=rubert_res, x='similarity', ax=ax[1])

In [ ]:
rubert_res.sort_values('similarity', ascending=False)

In [ ]:
res =
ozon_cats.merge(second_res, left_on='title', right_on='first_list', how='left').merge(wb_cats, left_on='second_list',
                                                                                      right_on='title',
                                                                                      how='left').rename(
    columns={'title_x': 'ozon',
             'id_x': 'id_ozon',
             'pid_x': 'pid_ozon',
             'title_y': 'wb',
             'id_y': 'id_wb',
             'pid_y': 'pid_wb'})
[['id_ozon', 'pid_ozon', 'ozon', 'id_wb', 'pid_wb', 'wb', 'similarity', 'group']].fillna(0).astype({'id_wb': 'int',
                                                                                                    'pid_wb': 'int'})

In [ ]:
res =
wb_cats.merge(second_res, left_on='title', right_on='second_list', how='left').merge(ozon_cats, left_on='first_list',
                                                                                     right_on='title',
                                                                                     how='right').rename(
    columns={'title_x': 'wb',
             'id_x': 'id_wb',
             'pid_x': 'pid_wb',
             'title_y': 'ozon',
             'id_y': 'id_ozon',
             'pid_y': 'pid_ozon'})[
    ['id_ozon', 'pid_ozon', 'ozon', 'id_wb', 'pid_wb', 'wb', 'similarity', 'group']].fillna(0).astype({'id_wb': 'int',
                                                                                                       'pid_wb': 'int'})

In [ ]:
res.loc[res['similarity'] > 1, 'similarity'] = 1

In [ ]:
def highlight(df):
    m1 = df['group'] == 0 & df['similarity'] == 0

    color = 'background-color: {}'.format

    style = pd.Series([color('')] * df.shape[0])
    style = style.mask(m1, color('orange'))
    style = pd.concat([style] * df.shape[1], axis=1)
    style.columns = df.columns
    style.index = df.index

    return style


In [ ]:
r = res.style.apply(highlight, axis=None)

# Классификация

In [8]:
mapped =
pd.read_excel('mapping_wb_ozon_full_17.11.xlsx', skiprows=10).drop(['Unnamed: 0', 'Unnamed: 7'], axis=1).rename(
    columns={'название категории': 'cat',
             'название параметра': 'param',
             'прямой мэтч': 'match',
             'id наиболее подходящей категории': 'similar_id',
             'название наиболее подходящего параметра': 'similar_param'})[
    ['charcId', 'similar_id', 'cat', 'param', 'match', 'similar_param']]

In [9]:
cat_param = mapped[['cat', 'param']].sort_values('cat').reset_index(drop=True)

In [10]:
res_df = pd.DataFrame()
for i in tqdm.notebook.tqdm(cat_param.index):
    temp_df = pd.DataFrame()
    temp = nlp(cat_param.iloc[i, 1])
    new_chunk_lemma = ' '.join(
        [token.lemma_ for token in temp if not token.is_punct and token.lemma_ not in nlp.Defaults.stop_words])
    has_null_vector = int(not all([token.vector_norm for token in nlp(new_chunk_lemma)]))

    if not has_null_vector:
        temp_df = pd.DataFrame({'cat': cat_param.iloc[i, 0],
                                'param': new_chunk_lemma},
                               index=[i])

    res_df = pd.concat([res_df, temp_df])

  0%|          | 0/5129 [00:00<?, ?it/s]

In [11]:
x = res_df['param']
y = res_df['cat']

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [13]:
x = np.array([x.vector for x in list(nlp.pipe(x))])
x_train = np.array([x.vector for x in list(nlp.pipe(x_train))])
x_test = np.array([x.vector for x in list(nlp.pipe(x_test))])

## Log reg

In [42]:
lr_clf = LogisticRegression(class_weight='balanced')
lr_clf.fit(x_train, y_train)

LogisticRegression(class_weight='balanced')

In [43]:
predicted = lr_clf.predict(x_test)

In [44]:
print(classification_report(y_test, predicted))

                                                   precision    recall  f1-score   support

                                         3D-ручки       0.00      0.00      0.00         1
                                       DVD-плееры       0.00      0.00      0.00         2
                                  FM-трансмиттеры       0.00      0.00      0.00         1
                                 Flash-накопители       0.00      0.00      0.00         5
                                       MP3 плееры       0.00      0.00      0.00         0
                               Powerline адаптеры       0.00      0.00      0.00         1
                                   RGBW усилители       0.00      0.00      0.00         3
                                   SSD-накопители       0.00      0.00      0.00         0
                                        Sim карты       0.00      0.00      0.00         2
                                       Web-камеры       0.50      1.00      0.67         

## RandomForest

In [46]:
forest_clf = RandomForestClassifier(n_estimators=100,
                                    bootstrap=True,
                                    max_features='sqrt')

forest_clf.fit(x_train, y_train)

RandomForestClassifier(max_features='sqrt')

In [49]:
rf_preds = forest_clf.predict(x_test)
rf_probs = forest_clf.predict_proba(x_test)[:, 1]